In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import sys, os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error

#user defined
from class_SurvivalQuilts import SurvivalQuilts
from utils_eval import calc_metrics

# IMPORT DATASET
    In this tutorial, preprocessed METABRIC dataset is used as a toy example.
    Please see the data type to use Survival Quilts on your own datasets.

In [3]:
#=================================================================#
##### USER-DEFINED FUNCTIONS
def f_get_Normalization(X, norm_mode):
    num_Patient, num_Feature = np.shape(X)

    if norm_mode == 'standard': #zero mean unit variance
        for j in range(num_Feature):
            if np.std(X[:,j]) != 0:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))/np.std(X[:,j])
            else:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))
    elif norm_mode == 'normal': #min-max normalization
        for j in range(num_Feature):
            X[:,j] = (X[:,j] - np.min(X[:,j]))/(np.max(X[:,j]) - np.min(X[:,j]))
    else:
        print("INPUT MODE ERROR!")

    return X
#=================================================================#
SEED=4321

In [7]:
# mit normalisation: funktioniert nur ohne CoxPH!!
file='/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb10.csv'
df = pd.read_csv(file, sep=',')

cols = ['int.rate', 'orig.upb', 'fico.score', 'dti.r',
       'ltv.r', 'bal.repaid', 't.act.12m', 't.del.30d.12m', 't.del.60d.12m',
       'hpi.st.d.t.o', 'hpi.zip.o', 'hpi.zip.d.t.o', 'ppi.c.FRMA',
       'TB10Y.d.t.o', 'FRMA30Y.d.t.o', 'ppi.o.FRMA', 'equity.est',
       'hpi.st.log12m', 'hpi.r.st.us', 'hpi.r.zip.st', 'st.unemp.r12m',
       'st.unemp.r3m', 'TB10Y.r12m', 'T10Y3MM', 'T10Y3MM.r12m']
    
label           = np.asarray(df[['default']])
time            = np.asarray(df[['time']])
df = df.drop(['label', 'payoff', 'current_year','default','time'], axis=1)
data            = np.asarray(df[cols])
data            = f_get_Normalization(data, 'standard')

X = pd.DataFrame(data,columns= cols)
T=  pd.DataFrame(time,columns =[['time']])
Y=  pd.DataFrame(label,columns =[['label']])
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

In [5]:
file = '/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb5.csv'
df = pd.read_csv(file, sep=',')
Y           = df[['default']]
T            = df[['time']]
df = df.drop(['label', 'payoff', 'current_year','default','time'], axis=1)
X = df
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]

eval_time_horizons

[14, 23, 36]

In [8]:
(T*30).max()

time    2160
dtype: int64

In [6]:
T

,time
0,7
1,20
2,25
3,20
4,3
...,...
9995,11
9996,15
9997,9
9998,44


In [39]:
# 1 = early repayment
SEED=4321
file='/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/meine/newdata_ER.csv'
df = pd.read_csv(file, sep=',')
Y  = df[['label']]
T  = df[['time']]
df = df.drop(['label', 'current_year','time'], axis=1)
X  = df
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

In [12]:
# 1 = early repayment NORMALIZATION
file='/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/meine/newdata_ER.csv'
df = pd.read_csv(file, sep=',')
label           = np.asarray(df[['label']])
time            = np.asarray(df[['time']])
df = df.drop(['label', 'current_year','time'], axis=1)
cols = df.columns
data            = np.asarray(df[cols])
data            = f_get_Normalization(data, 'standard')

X = pd.DataFrame(data,columns= cols)
T=  pd.DataFrame(time,columns =[['time']])
Y=  pd.DataFrame(label,columns =[['label']])

eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

In [23]:
#mortage NORMALIZATION
file = '/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/WideFormatMortgageAfterRemovingNull.csv'
df = pd.read_csv(file, sep=',')

label           = np.asarray(df[['default_time']])
time            = np.asarray(df[['duration']])
df = df.drop(["id", "first_time", "payoff_time", "status_time", "time",'duration','default_time'], axis=1)
cols=df.columns
data            = np.asarray(df[cols])
data            = f_get_Normalization(data, 'standard')

X = pd.DataFrame(data,columns= cols).iloc[:5000,]
T=  pd.DataFrame(time,columns =[['time']]).iloc[:5000,]
Y=  pd.DataFrame(label,columns =[['label']]).iloc[:5000,]

eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

In [31]:
#mortage
file = '/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/WideFormatMortgageAfterRemovingNull.csv'
df = pd.read_csv(file, sep=',')

Y           = df[['default_time']].iloc[:5000,]
T            = df[['duration']].iloc[:5000,]
df = df.drop(["id", "first_time", "payoff_time", "status_time", "time",'duration','default_time'], axis=1)
X           = df.iloc[:5000,]
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]
tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

In [32]:
X.info()
Y[Y==1].describe(),Y[Y==0].describe(),Y.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   orig_time                5000 non-null   int64  
 1   mat_time                 5000 non-null   int64  
 2   balance_time             5000 non-null   float64
 3   LTV_time                 5000 non-null   float64
 4   interest_rate_time       5000 non-null   float64
 5   hpi_time                 5000 non-null   float64
 6   gdp_time                 5000 non-null   float64
 7   uer_time                 5000 non-null   float64
 8   REtype_CO_orig_time      5000 non-null   int64  
 9   REtype_PU_orig_time      5000 non-null   int64  
 10  REtype_SF_orig_time      5000 non-null   int64  
 11  investor_orig_time       5000 non-null   int64  
 12  balance_orig_time        5000 non-null   float64
 13  FICO_orig_time           5000 non-null   int64  
 14  LTV_orig_time           

(       default_time
 count        1804.0
 mean            1.0
 std             0.0
 min             1.0
 25%             1.0
 50%             1.0
 75%             1.0
 max             1.0,
        default_time
 count        3196.0
 mean            0.0
 std             0.0
 min             0.0
 25%             0.0
 50%             0.0
 75%             0.0
 max             0.0,
        default_time
 count   5000.000000
 mean       0.360800
 std        0.480281
 min        0.000000
 25%        0.000000
 50%        0.000000
 75%        1.000000
 max        1.000000)

In [4]:
file

'/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb10.csv'

# TRAIN SURVIVAL QUILTS

In [5]:
model_sq = SurvivalQuilts(K=6,num_bo=10, num_cv=5,num_outer=1,step_ahead=1) #K=2,num_bo=5, num_cv=5,num_outer=3 :ndb9.cv
model_sq.train(tr_X, tr_T, tr_Y) 
#best ndb9.cv: K=5,num_bo=75, num_cv=5,num_outer=1 models:
#['CoxPH', 'CoxPHRidge', 'LogNormal', 'LogLogistic']

# save model
#import pickle 
#filename = './results/' + 'SurvivalQuilts.sav'
#pickle.dump(model_sq, open(filename, 'wb'))

initial training of underlying models...
CV.. 1/5
CV.. 2/5
CV.. 3/5
CV.. 4/5
CV.. 5/5
TIME K = 0
[[0.99999999 0.         0.         0.         0.         0.        ]
 [0.99999999 0.         0.         0.         0.         0.        ]
 [0.99999999 0.         0.         0.         0.         0.        ]
 [0.99999999 0.         0.         0.         0.         0.        ]
 [0.99999999 0.         0.         0.         0.         0.        ]
 [0.99999999 0.         0.         0.         0.         0.        ]]
[[0.99999999 0.         0.         0.         0.         0.        ]]
[[-0.65285632]]
=========== BO Finished ===========
!X_step_ens aus dem W gewählt wird:  [[0.99999999 0.         0.         0.         0.         0.        ]
 [0.01428985 0.14846811 0.15378349 0.23555761 0.23580347 0.21209747]
 [0.17938163 0.16766643 0.13544851 0.18059268 0.17070634 0.16620442]
 [0.00946936 0.21334632 0.2081924  0.22186073 0.18981076 0.15732043]
 [0.2081137  0.23704934 0.01186351 0.23589829 0.07538

[[0.         0.         0.         0.         0.         0.99999999]]
[[-0.40701055]]
=========== BO Finished ===========
!X_step_ens aus dem W gewählt wird:  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 9.99999990e-01]
 [2.26849853e-01 2.31367162e-01 2.13993870e-01 2.36280982e-01
  6.38281023e-02 2.76800291e-02]
 [2.18331355e-01 1.80368716e-01 1.44366690e-01 2.01351843e-01
  2.16172507e-01 3.94088861e-02]
 [1.79911301e-01 1.17280594e-01 1.82866653e-01 1.77925839e-01
  1.67656702e-01 1.74358909e-01]
 [3.38209928e-01 3.40565993e-03 3.36696341e-01 2.20952914e-01
  9.99643122e-02 7.70841324e-04]
 [2.72612769e-01 2.39625985e-01 7.81548949e-02 6.45894687e-02
  1.15855848e-01 2.29161030e-01]
 [1.79632994e-01 1.82306431e-01 1.89815867e-01 1.68406789e-01
  1.29309459e-01 1.50528458e-01]
 [1.57871461e-01 1.95204393e-01 1.75652535e-01 1.95418412e-01
  9.23507276e-02 1.83502470e-01]
 [9.78484896e-02 1.72225111e-01 1.86244437e-01 1.75839606e-01
  1.85737790e-01 1.

# TEST SURVIVAL QUILTS

In [6]:
pred = model_sq.predict(te_X, eval_time_horizons)

for e_idx, eval_time in enumerate(eval_time_horizons):
    print( calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, e_idx], eval_time) )
    
print(model_sq.model_names)

(0.9952286370331166, 0.36276305273658005)
(0.9942456538707999, 0.5969161731039037)
(0.9892782969388593, 0.7539280878582325)
['CoxPH', 'CoxPHRidge', 'Weibull', 'LogNormal', 'LogLogistic', 'RandomSurvForest']


In [7]:
#pred = model_sq.predict(te_X, eval_time_horizons)
#self.model_names   = ['CoxPH', 'Weibull', 'LogNormal']
#for e_idx, eval_time in enumerate(eval_time_horizons):
#print( calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, e_idx], eval_time) )
model_sq.quilting_patterns

array([[0.99999999, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.99999999]])

In [8]:
for m in range(model_sq.M):
    tmp_pred_eval = model_sq.underlying_models[m].predict(te_X, eval_time_horizons)
    #Name des Modells:
    a = str(model_sq.underlying_models[m])
    b = a.split(".")
    b = " ".join(b[1:])
    c = b.split()[0]
    print(c) #Name des Modells
    
    if model_sq.quilting_patterns[0][m] > 0.5: #first time check
        #print('For time = ',eval_time,'best weight in this model (',model_sq.quilting_patterns[e_idx][m],')\n')
        tmp_pred_LN=tmp_pred_eval
        name = c
    if (model_sq.quilting_patterns[0][m] > 0.) & (model_sq.quilting_patterns[0][m] <0.5):
        print('->',model_sq.quilting_patterns[e_idx][m])

    for e_idx, eval_time in enumerate(eval_time_horizons):
        print( calc_metrics(tr_T, tr_Y, te_T, te_Y, tmp_pred_eval[:, e_idx], eval_time) )
    print('\n')
print(np.sum(np.sqrt((pred-tmp_pred_LN)**2)))    
mean_squared_error(pred,tmp_pred_LN)     

CoxPH
(0.9953472437106707, 0.36331086999949197)
(0.5, 0.3853559067867073)
(0.5, 0.1571754462655214)


CoxPHRidge
(0.9958231476007862, 0.3626635805244224)
(0.5, 0.3853559067867073)
(0.5, 0.1571754462655214)


Weibull
(0.9923062104369722, 0.36245067731058545)
(0.9874196282941636, 0.5967190688090201)
(0.980555388717425, 0.7575686728721543)


LogNormal
(0.9912633248454059, 0.3624349745883508)
(0.9911167309052896, 0.5960532511775498)
(0.98755219172386, 0.7546588210085543)


LogLogistic
(0.9932280762225829, 0.36282875407602305)
(0.9917407962387448, 0.5969667376502096)
(0.9871465475219785, 0.7562406856933871)


RandomSurvForest
(0.9947615992001357, 0.36333412921446295)
(0.9948587601961653, 0.5967087492429268)
(0.990440953052584, 0.7495331909388094)


3950.412891262829


0.640433487054831

In [9]:
for t in range(len(eval_time_horizons)):
    print('\n',"time: ", eval_time_horizons[t])#," metric_CINDEX",'\n' )

    print(name,"         ",calc_metrics(tr_T, tr_Y, te_T, te_Y, tmp_pred_LN[:, t], eval_time_horizons[t]))
    print("Temporal Quilting"," ",calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, t], eval_time_horizons[t]))


 time:  14
CoxPH           (0.9953472437106707, 0.36331086999949197)
Temporal Quilting   (0.9952286370331166, 0.36276305273658005)

 time:  22
CoxPH           (0.5, 0.3853559067867073)
Temporal Quilting   (0.9942456538707999, 0.5969161731039037)

 time:  34
CoxPH           (0.5, 0.1571754462655214)
Temporal Quilting   (0.9892782969388593, 0.7539280878582325)


In [12]:
file

'/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/hu_gitHub/ndb9.csv'

In [13]:
idx=10
end = idx+40
for y, pr in zip(te_Y.iloc[idx:end,0],pred[idx:end,:]):
    print('Time:  ', eval_time_horizons)
    print(y,pr)#, '\n')
    if y==1:
        print('time of default: ',te_T.iloc[idx,0])
    print()
    idx+=1
pred.shape

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0.08 0.09 0.14]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0.   0.   0.01]

Time:   [12, 21, 31]
0 [0.   0.   0.01]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0.   0.03 0.04]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0.   0.01 0.03]

Time:   [12, 21, 31]
0 [0.   0.   0.04]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
1 [0.0025     0.09       0.28191666]
time of default:  28

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
1 [0.53866666 0.86491666 0.91991666]
time of default:  11

Time:   [12, 21, 31]
0 [0.03       0.23916666 0.31666666]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [12, 21, 31]
0 [0. 0. 0.]

Time:   [1

(2000, 3)

In [ ]:
(0.9927828026964547, 0.2641950444596503)
(0.9592583449503984, 0.5217450168586214)
(0.9751791817675914, 0.769228162630255)

In [102]:
#=================================================================#
##### USER-DEFINED FUNCTIONS
#def f_get_Normalization(X, norm_mode):
    num_Patient, num_Feature = np.shape(X)

    if norm_mode == 'standard': #zero mean unit variance
        for j in range(num_Feature):
            if np.std(X[:,j]) != 0:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))/np.std(X[:,j])
            else:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))
    elif norm_mode == 'normal': #min-max normalization
        for j in range(num_Feature):
            X[:,j] = (X[:,j] - np.min(X[:,j]))/(np.max(X[:,j]) - np.min(X[:,j]))
    else:
        print("INPUT MODE ERROR!")

    return X
#=================================================================#



##### DATASET SELECTION
SEED    = 1111
data_mode = 'metabric'  #{'metabric', 'support'}
mode = 'meine'
        
if data_mode == 'metabric':
    if mode == 'meine':
        #X       = pd.read_csv('/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/meine/newdata_x_bad.csv')
        tmp     = pd.read_csv('/Users/svpovol/Desktop/Bachelorarbeit/main_ordner/Data/meine/newdata_bad.csv')
        T       = tmp[['time']]
        Y       = tmp[['label']]
        to_remove = ['t.act.12m', 'hpi.st.d.t.o'] #strong correlated features
        #X = X.drop(to_remove, axis = 1)           #remove it
        #f_get_Normalization:
        #cols = X.columns.tolist()
        #data            = f_get_Normalization(np.asarray(X[cols]), 'standard') 
        #X = pd.DataFrame(data,columns= cols)
    else:
        X       = pd.read_csv('./sample data/metabric_cleaned_features_final.csv')
        tmp     = pd.read_csv('./sample data/metabric_label.csv')
        T       = tmp[['event_time']]
        Y       = tmp[['label']]
    #time_interval_ = 10.
    SEED    = 4321
    if X.shape[0]>1215000:     #same size as my dataset
        print(X.shape)
        X= X.iloc[0:1215,:]
        T= T.iloc[0:1215,:]
        Y= Y.iloc[0:1215,:] 

tmp_folder = './results/' + data_mode + ' (seed ' + str(SEED) +')/'
if not os.path.exists(tmp_folder):
    os.makedirs(tmp_folder)

# eval_time_horizons can be selected based on one's interest.
eval_time_horizons = [int(T[Y.iloc[:,0] == 1].quantile(0.25)), int(T[Y.iloc[:,0] == 1].quantile(0.50)), int(T[Y.iloc[:,0] == 1].quantile(0.75))]


tr_X,te_X, tr_T,te_T, tr_Y,te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)
#eval_time_horizons =[11, 19, 30,48]#= [24, 48, 72]
#T.shape,X.columns,X

T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   time    1215 non-null   int64
dtypes: int64(1)
memory usage: 9.6 KB


In [14]:
pred_all = model_sq._get_ensemble_prediction(model_sq.underlying_models, model_sq.quilting_patterns, te_X, model_sq.all_time_horizons)


In [16]:
pred_all[3,60]

0.23999999760000004

In [15]:
te_X.shape

(2000, 25)

In [ ]:
pred_all[:, np.where(np.asarray(model_sq.all_time_horizons) <= eval_time)[0][-1]]

In [30]:
np.where(np.asarray(model_sq.all_time_horizons) <= 21)[0][-1]

18

In [27]:
model_sq.all_time_horizons[21]

24

In [31]:
W_ =model_sq.quilting_patterns
var = model_sq.all_time_horizons # model_sq.time_horizons
for m in range(model_sq.M):
            tmp_pred_ = model_sq.underlying_models[m].predict(te_X, var)

            if m == 0:
                pred_ = np.zeros(np.shape(tmp_pred_))
            #else:
            for tt in range(model_sq.K):
                print(m,tt,'\n')
                if tt == 0:
                    tmp_time_idx1 = np.asarray(var) <= model_sq.time_horizons[tt]
                    #print(tmp_time_idx1)
                    
                    #print(tmp_time_idx1,'\n',tmp_pred_[:2, tmp_time_idx1],'\n',tmp_pred_[:2, tmp_time_idx1][:,[0]],'\n',np.ones([1,np.sum(tmp_time_idx1)]))
                    
                    tmp_time_idx2 = np.asarray(var) > model_sq.time_horizons[tt]
                    #print(tmp_time_idx2)
                    #print(tmp_time_idx2,'\n',tmp_pred_[:2, tmp_time_idx2])

                    increment = tmp_pred_[:, tmp_time_idx1] - np.matmul(tmp_pred_[:, tmp_time_idx1][:,[0]], np.ones([1,np.sum(tmp_time_idx1)]))
                    print(increment[:2,:])

                    pred_[:, tmp_time_idx1] =  pred_[:, tmp_time_idx1] + W_[tt,m] * increment
                    pred_[:, tmp_time_idx2] =  pred_[:, tmp_time_idx2] + W_[tt,m] * np.matmul(increment[:,[-1]], np.ones([1,np.sum(tmp_time_idx2)]))
                elif tt == (model_sq.K - 1): #the last index  
                    tmp_time_idx1 = np.asarray(var) > model_sq.time_horizons[tt-1]

                    increment = tmp_pred_[:, tmp_time_idx1] - np.matmul(tmp_pred_[:, tmp_time_idx1][:,[0]], np.ones([1,np.sum(tmp_time_idx1)]))
                    print(increment[:2,:])

                    pred_[:, tmp_time_idx1] =  pred_[:, tmp_time_idx1] + W_[tt,m] * increment

                else:
                    tmp_time_idx1 = (np.asarray(var) > model_sq.time_horizons[tt-1]) & (np.asarray(var) <= model_sq.time_horizons[tt])
                    tmp_time_idx2 = np.asarray(var) > model_sq.time_horizons[tt]

                    increment = tmp_pred_[:, tmp_time_idx1] - np.matmul(tmp_pred_[:, tmp_time_idx1][:,[0]], np.ones([1,np.sum(tmp_time_idx1)]))
                    print(increment[:2,:],'\n')

                    pred_[:, tmp_time_idx1] =  pred_[:, tmp_time_idx1] + W_[tt,m] * increment
                    pred_[:, tmp_time_idx2] =  pred_[:, tmp_time_idx2] + W_[tt,m] * np.matmul(increment[:,[-1]], np.ones([1,np.sum(tmp_time_idx2)]))



0 0 

[[0.00000000e+00 7.18460499e-06 1.80527725e-05 2.52893440e-05]
 [0.00000000e+00 1.68333595e-03 4.22435543e-03 5.91271725e-03]]
0 1 

[[0.00000000e+00 1.22914465e-05 3.60378476e-05 5.53487808e-05
  9.36985891e-05 1.79491524e-04 2.19761956e-04 2.68804347e-04
  3.15096157e-04 4.33654231e-04 4.95721520e-04]
 [0.00000000e+00 2.85435196e-03 8.34563077e-03 1.27888034e-02
  2.15533062e-02 4.08783064e-02 4.98164805e-02 6.05885858e-02
  7.06438291e-02 9.59054403e-02 1.08853676e-01]] 

0 2 

[[0.00000000e+00 2.01344499e-04 5.10183640e-04 9.97220084e-04
  1.30502432e-03 1.55809651e-03 2.03404313e-03 2.94159551e-03
  3.86577114e-03 4.47703476e-03 7.23095854e-03 1.17949418e-02]
 [0.00000000e+00 3.93317036e-02 9.61739044e-02 1.77868714e-01
  2.24896868e-01 2.61102260e-01 3.23642638e-01 4.25184603e-01
  5.08600741e-01 5.54622710e-01 6.96552172e-01 7.98902952e-01]] 

0 3 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

0 4 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 